## Importing packages

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import os
from skimage.io import imread
from skimage.color import rgb2lab, lab2rgb
from PIL import Image
from keras.preprocessing.image import  img_to_array, load_img
from sklearn.model_selection import train_test_split

2022-03-19 11:52:52.145117: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-19 11:52:52.145135: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
NUM_OF_IMAGES=10

## Preparing the dataset
    1.We load our images from the given directory
    2.We convert our images to numpy arrays


In [ ]:
# dataset directory
direc='C:/Users/2019/Desktop/Studying/Machine Learning/Image_recolorizer/tiny-imagenet-200/train/n01443537/images2' 

# counter to limit the amount of images we read
counter=0

images_to_arrays=[]
img_input=[]

for x in os.listdir(direc):
    # we only read the first 'counter' amount of Jpegs
    if x.endswith('.JPEG') and counter<10:      
        counter+=1
        # Generating the corrosponsing path to the picture
        path=direc+'/'+x
        target_size=(64,64)
        # loading images from the path
        img_input.append(load_img(path,target_size=target_size))
        
# Converting all the pictures to numpy arrays       
for x in img_input:
    #print(img_to_array(x).shape)
    images_to_arrays.append(img_to_array(x))          
images_to_arrays=np.array(images_to_arrays)
# print(images_to_arrays.shape)

### Side Functions:
    1.We define plotChannels function to assist in plotting the bounds/ranges of each of the channel types
    2.We define L_extract to extract the different channels from an image's LAB version

In [ ]:
def plotChannels(images_to_arrays,labels=['r','g','b']):
    #going over each label and calculating max/min values
    for x,y in enumerate(labels):
        mi=np.min(images_to_arrays[:,:,:,x])
        ma=np.max(images_to_arrays[:,:,:,x])
        print('{}: min={:8.4f}, max={:8.4f}'.format(y,mi,ma))

plotChannels(images_to_arrays,labels=['r','g','b'])

In [ ]:
def L_extract(img,dim):
    result=np.zeros(img.shape)
    # we return the given channel of LAB in rgb and adjust the brightness only on A and B
    if dim != 0:
        result[:,:,0]=60
    result[:,:,x]=img[:,:,dim]
    #convert back to rgb to plot
    result=lab2rgb(y)
    return result

In [ ]:
count=1
plut=plt.figure(figsize=(12,3*images_to_arrays.shape[0]))
for channel in images_to_arrays:
    #adding the subplot for the original picture
    x=plut.add_subplot(images_to_arrays.shape[0],4,count)
    x.imshow(channel/255.0); x.axis('off')
    count+=1
    
    for dim,label in enumerate(['r','g','b']):
        #adding the subplot for each channel for the original picture
        uni_colored=np.zeros(channel.shape)
        #we set each channel excluding the one we want to 0
        uni_colored[:,:,dim]=channel[:,:,dim]
        x=plut.add_subplot(images_to_arrays.shape[0],4,count)
        x.imshow(uni_colored/255.0); x.axis('off')
        count+=1
    

## Converting RGB to LAB

In [ ]:
normalized_images=images_to_arrays/255.0
plotChannels(normalized_images,['r','g','b'])
img_arr_lab=rgb2lab(normalized_images)
plotChannels(img_arr_lab,['l','a','b'])

In [ ]:
img_lab_rgb=np.zeros(img_arr_lab.shape)
img_lab_rgb=lab2rgb(img_arr_lab)
plotChannels(img_lab_rgb.reshape((1,)+img_lab_rgb.shape),['r','g','b'])

In [ ]:
count=1
fig=plt.figure(figsize=(6,3*images_to_arrays.shape[0]))
#we enumerate over our original images and images after double conversion to compare the loss
for irgb,irgb2 in zip(normalized_images,img_lab_rgb):
    #original
    x=fig.add_subplot(images_to_arrays.shape[0],2,count)
    x.imshow(irgb); x.axis('off')
    x.set_title('rgb')
    count+=1
    #doubly converted
    x=fig.add_subplot(images_to_arrays.shape[0],2,count)
    x.imshow(irgb2); x.axis('off')
    x.set_title('rgb2lab2rgb')
    count+=1
    
plt.show()
    

In [ ]:
count=1
fig=plt.figure(figsize=(13,3*images_to_arrays.shape[0]))
#we plot the three channels of the LAB versions of our images
for img in img_arr_lab:
    #L
    x=fig.add_subplot(images_to_arrays.shape[0],3,count)
    lab_rgb_dim=L_extract(img,0)
    x.imshow(lab_rgb_dim) ; x.axis('off')
    x.set_title('L lmao')
    count+=1
    #A
    x=fig.add_subplot(images_to_arrays.shape[0],3,count)
    lab_rgb_dim=L_extract(img,1)
    x.imshow(lab_rgb_dim) ; x.axis('off')
    x.set_title('A')
    count+=1
    #B
    x=fig.add_subplot(images_to_arrays.shape[0],3,count)
    lab_rgb_dim=L_extract(img,2)
    x.imshow(lab_rgb_dim) ; x.axis('off')
    x.set_title('B')
    count+=1

plt.show()

In [ ]:
L_array=[]
A_array=[]
B_array=[]
#we extract each dimension of the LAB images onto a different array
for img in img_arr_lab:
    L_array.append(L_extract(img,0))
    A_array.append(L_extract(img,1))
    B_array.append(L_extract(img,2))

#we insert our data into x and y such that x is our gray images (features), y is our A and B images (classes)
y=np.hstack((A_array,B_array)).reshape(NUM_OF_IMAGES,2,64,64,3)
x=L_array
print(LAB_arr.shape)

In [ ]:
#we split into train and test such that train=80% * overalldata
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
batch_size=128
epochs=10
input_shape=(64,64,1)